This notebook will calculate new crashes based only on existing crashes, so it is consistent with whatever type/source of existing crash data the tool uses, and is not dependent on the current implementation of the crashes model

This is the equation being implemented:
$NC_{cmojk}=EC_{cmoj} * (1 + \sum_{i}\sum_{F}E_{ik} * \frac{Ni}{L} * I_{F})^{p}*CRF_{mojk}$

In [ ]:
import numpy as np
import pandas as pd

In [ ]:
existing_crashes = pd.read_csv('output_2023_09_05/reports/safety-4-combined-b-crashes-all.csv')
infrastructure = pd.read_csv('output_2023_09_05/reports/overall-4-infrastructure-safety.csv')
infrastructure_volume_changes = pd.read_csv('output_2023_09_05/lookups/per_element_travel_adjustments.csv')

In [ ]:
## Pull out the required variables: element, crashes volume increase per element, element share, improvement type, CRF
CRFmojk = existing_crashes["CRFmojk"]
element = infrastructure["Infrastructure type"]
element = "conventional-bike-lane"
share = infrastructure["Project share"]
## Fix later

In [ ]:
## testing
volume_change = infrastructure_volume_changes[infrastructure_volume_changes["element"] == element]
volume_change.reset_index().at[0,"mean adjustment (%)"]

In [ ]:
## Define the function
## have to iterate through all infrastructure for the one project ID
def new_NC(Project_ID, Estimate, EC, CRF):
    project_elements = infrastructure[infrastructure["Project ID"] == Project_ID]
    volume_change_factor = 1
    for index, element in project_elements.iterrows():
        element_name = element["Infrastructure type"]
        share = element["Project share"]
        improvement_type = element["Improvement type"]
        if improvement_type == "retrofit":
            improvement_factor = 0.1
        else:
            improvement_factor = 1
        sel_volume_change = infrastructure_volume_changes[infrastructure_volume_changes["element"] == element_name]
        if len(sel_volume_change) == 0:
            volume_change = 0
        else:
            volume_change = (sel_volume_change.reset_index().at[0, Estimate + " adjustment (%)"])/100
        volume_change_factor += volume_change * share * improvement_factor
    ## volume change factor raised to "safety in numbers" power
    volume_change_factor = (volume_change_factor)**0.5
    NC = EC * volume_change_factor * CRF
    return NC
## I know this is not going to work because pandas iterates over columns, not rows, but just sketching out an idea
## Terrible and messy code
## Fix later - this is not the final implementation for the tool anyway, just testing to see if the equation improves the results

In [ ]:
def apply_new_NC(element,EC_type):
    ## request the EC_type (eg what was used in the tool) to be added as a column to this table
    ## then use whatever the tool used for the NC_new column
    Project_ID = element["Project ID"]
    EC = element["ECmoj" + EC_type]
    Estimate = element["K estimate"]
    CRF = element["CRFmojk"]
    NC = new_NC(Project_ID,Estimate,EC,CRF)
    return(NC)

In [ ]:
for index, row in existing_crashes.iterrows():
    existing_crashes.at[index,"NC_user"] = apply_new_NC(row," with user input")
    existing_crashes.at[index,"NC_model"] = apply_new_NC(row," model")

### Calculate Crash Change

In [ ]:
## Calculate crash change: NC - EC for both model and user input (later add "whatever the tool used" version)
existing_crashes["Change_user"] = existing_crashes["NC_user"] - existing_crashes["ECmoj with user input"]
existing_crashes["Change_model"] = existing_crashes["NC_model"] - existing_crashes["ECmoj model"]

In [ ]:
total_crash_change = pd.read_csv('output_2023_09_05/reports/safety-4-combined-c-crashes-volume.csv')

In [ ]:
## Sum crashes by location type - maintain split by project, mode, outcome, estimate
change_model_mok = existing_crashes.groupby(["Project ID","M Mode","O Outcome","K estimate"])["Change_model"].sum()
change_user_mok = existing_crashes.groupby(["Project ID","M Mode","O Outcome","K estimate"])["Change_user"].sum()
change_user_mok.loc[("644adafaab814ec4fdd30fab","bicycling","crash","lower")]

In [ ]:
## Try to add Crash_change_model and Crash_change_user to the total_crash_change table
for index, row in total_crash_change.iterrows():
    ## The set of characteristics for this project, mode, outcome, estimate
    row_chars = (row["Project ID"],row["M Mode"],row["O Outcome"],row["K Estimate"])
    ## The corresponding model and user crash changes (summed by location type in the previous cell)
    total_crash_change.at[index,"model_crash_change"] = change_model_mok.loc[row_chars]
    total_crash_change.at[index,"user_crash_change"] = change_user_mok.loc[row_chars]

### Calculate Relative Crashes
- Before crashes per 1000 volume: user
- Before crashes per 1000 volume: model
- After crashes per 1000 volume: user
- After crashes per 1000 volume: model
- Change in crashes per 1000 volume: user (after - before)
- Change in crashes per 1000 volume: model (after - before)

In [ ]:
## Get existing and projected volume from safety-5-volume-d-combined.csv
volume = pd.read_csv('output_2023_09_05/reports/safety-5-volume-d-combined.csv')

In [ ]:
for index, volume_row in volume.iterrows():
    projectID = volume_row["Project ID"]
    m = volume_row["M Mode"]
    j = volume_row["J Location type"]
    k = volume_row["K Estimate"]
    crashes_rows = existing_crashes[existing_crashes["Project ID"] == projectID][existing_crashes["M Mode"] == m][existing_crashes["J Location"] == j][existing_crashes["K estimate"] == k]
    for index, crashes_row in crashes_rows.iterrows():
        if (volume_row["Existing Volume Vmj"] == 0):
            # print("Error: 0 existing volume in %s"% [projectID, m, crashes_row["O Outcome"], j, k])
            # commenting out for now to speed this up
            total_crash_change.at[index, "before_model"] = 'NaN'
            total_crash_change.at[index, "before_user"] = 'NaN'
            # Is using the index like this a valid way to do this?
        else:
            total_crash_change.at[index, "before_model"] = crashes_row["ECmoj model"]/volume_row["Existing Volume Vmj"] * 1000
            total_crash_change.at[index, "before_user"] = crashes_row["ECmoj with user input"]/volume_row["Existing Volume Vmj"] * 1000
        if (volume_row["Projected Volume PVmjk"] == 0):
            # print("Error: 0 projected volume in %s"% [projectID, m, crashes_row["O Outcome"], j, k])
            total_crash_change.at[index, "after_model"] = 'NaN'
            total_crash_change.at[index, "after_user"] = 'NaN'
        else:
            total_crash_change.at[index, "after_model"] = crashes_row["NC_model"]/volume_row["Projected Volume PVmjk"] * 1000
            total_crash_change.at[index, "after_user"] = crashes_row["NC_user"]/volume_row["Projected Volume PVmjk"] * 1000
total_crash_change
## Wow, that is a lot of projects with zero volume! need to look into this later!!
## Also seeing projects with 'NaN' volume - what is happening with these projects?

### Recalculate with "fixed" crash model
Now that we have identified a prominent error with the model and calculated new accurate values (Individual_segment_intersection_crash_model.ipynb), let's use those values to calculate model NC and crash change with these equations again
$EC_{cmoj} = (EV_{cmj})^{p} * \sum_{f}\sum_{v}e^{Ɑ_{mojvf}} * L_{jvf}$

In [ ]:
# The easiest approach I can think of right now is to export those results as a CSV and import here
Ljvf=pd.read_csv('output_2023_09_05/reports/overall-3-reach-Ljvf.csv')
alpha=pd.read_csv('output_2023_09_05/lookups/alpha.csv')

## 1. setup - combine Ljvf and alpha
alpha["e_alpha"] = np.exp(alpha["alpha"])
alpha = alpha.rename(columns={"volume":"V volume class","functional class":"F functional class","location type":"J location type","mode":"M Mode","outcome":"O Outcome"})
Ljvf_alpha = pd.merge(Ljvf,alpha,on=["V volume class","F functional class","J location type"])

## 2. sum of ljvf * e^amojvf over v and f (keep separated by m,o,j)
Ljvf_alpha["Ljvf_e_alpha"] = Ljvf_alpha["Count or length"]*Ljvf_alpha["e_alpha"]
Lj_alpha = Ljvf_alpha.groupby(["Project ID","M Mode","O Outcome","J location type"]).sum()

## 3. multiply by Vmj
volume = volume.rename(columns={"J Location type":"J location type"})
Lj_alpha = Lj_alpha.reset_index()
Lj_alpha_volume = pd.merge(Lj_alpha,volume,how='outer',on=["Project ID","M Mode","J location type"])
(Lj_alpha_volume["Existing Volume Vmj"]*Lj_alpha_volume["Ljvf_e_alpha"]).plot()
## Why are these so large??

In [ ]:
Ljvf_alpha[(Ljvf_alpha["J location type"] == "intersection") & (Ljvf_alpha["Ljvf_e_alpha"] > 2)]

In [ ]:
Ljvf[(Ljvf["Project ID"] == "6490d4551930d10600997fd1") & (Ljvf["J location type"] == "roadway")]

In [ ]:
Ljvf_alpha[(Ljvf_alpha["Project ID"] == "6490d4551930d10600997fd1") & (Ljvf_alpha["J location type"] == "roadway")]

In [ ]:
Ljvf_alpha[(Ljvf_alpha["Project ID"] == "6490d4551930d10600997fd1") & (Ljvf_alpha["J location type"] == "roadway")].groupby(["J location type","M Mode","O Outcome"]).sum()

In [ ]:
Lj_alpha_volume[(Lj_alpha_volume["K Estimate"] == "mean") & (Lj_alpha_volume["O Outcome"] == "crash")]

In [ ]:
volume["Existing Volume Vmj"].plot()
## What?? Why are the volumes so large? This seems off somehow?
segments = pd.read_csv('output_2023_09_05/reports/overall-5-ways.csv')
intersections = pd.read_csv('output_2023_09_05/reports/overall-6-intersections.csv')

segments_n = segments.replace(["Not applicable","Not available"],np.NaN)
intersections_n = intersections.replace(["Not applicable","Not available"],np.NaN)

segments_n["Bicycle exposure"]=pd.to_numeric(segments_n["Bicycle exposure"])
segments_n["Pedestrian exposure"]=pd.to_numeric(segments_n["Pedestrian exposure"])
intersections_n["Bicycle exposure"]=pd.to_numeric(intersections_n["Bicycle exposure"])
intersections_n["Pedestrian exposure"]=pd.to_numeric(intersections_n["Pedestrian exposure"])

segments_n.groupby("Project ID").sum().plot()
intersections_n.groupby("Project ID").sum().plot()

# OK I guess it is about the same as what I got from the individual segments/intersections then
# It does look slightly larger - 800,000 max instead of 700,000 max? 

In [ ]:
## What about the e^alpha constant * Ljvf? Is that the same as the individual segments method?

In [ ]:
### Ohh, it's probably just the ^0.5 power for the volume
#(pow(Lj_alpha_volume["Existing Volume Vmj"],0.5)*Lj_alpha_volume["Ljvf_e_alpha"]).plot()
Lj_alpha_volume["Crashes"] = (pow(Lj_alpha_volume["Existing Volume Vmj"],0.5)*Lj_alpha_volume["Ljvf_e_alpha"])
Lj_alpha_volume.groupby(["M Mode","J location type"])["Crashes"].plot(legend=True)
# Lj_alpha_volume["Crashes"] = pow(Lj_alpha_volume["Existing Volume Vmj"],0.5)*Lj_alpha_volume["Ljvf_e_alpha"]
# Lj_alpha_volume[Lj_alpha_volume["Crashes"] > 800]
## No, this is better but still basically what the tool is outputting (max 1400 crashes)
## So what is the real difference between calculating at separate intersections/segments and calculating in the aggregate???
## Wait, what happened to the "outcome" column? Anyway clearly I am either making a mistake or the whole equation is flawed in the first place.

In [ ]:
# Compared to the existing model crashes from the tool - basically looks the same? Why?
existing_crashes["ECmoj model"].plot(legend=True)

Uh oh, it looks like this method (multiply volume outside of the summation) doesn't actually fix the issue? Why does it work for adding up the segments/intersections separately then?

So looking more carefully at the equation (see github issue) the implementation actually does need to use separated volume by volume/functional class
Correct implementation: $EC_{cmoj} =  \sum_{f}\sum_{v}e^{Ɑ_{mojvf}} * L_{jvf} * (EV_{cmjvf})^{p}$

In [ ]:
# Convert to feet
segments_n["Length_count"] = segments_n["Length"]/5280
# Find totals for each volume/functional class
segments_volume = segments_n.groupby(["Project ID","Bicycle volume class","Functional class"]).sum().reset_index()
# "Count" of one intersection = 1
intersections_n.loc[:,"Length_count"] = 1
# Find totals for each volume/functional class
intersections_volume = intersections_n.groupby(["Project ID","Pedestrian volume class","Functional class"]).sum().reset_index()

In [ ]:
# Set the location type to be able to match up with the Ljvf/alpha constant tables
segments_volume.loc[:,"J location type"] = "roadway"
intersections_volume.loc[:,"J location type"] = "intersection"

In [ ]:
# Rename and change capitalization to be able to merge
segments_volume["Bicycle volume class"] = segments_volume["Bicycle volume class"].str.lower()
segments_volume = segments_volume.rename(columns={"Bicycle volume class":"V volume class","Functional class":"F functional class"})
intersections_volume["Pedestrian volume class"] = intersections_volume["Pedestrian volume class"].str.lower()
intersections_volume = intersections_volume.rename(columns={"Pedestrian volume class":"V volume class","Functional class":"F functional class"})

In [ ]:
# Combine volume for segments and intersections
volume_mjvf = pd.concat([intersections_volume,segments_volume])

In [ ]:
# Combine with Ljvf and alpha constants
Ljvf_alpha_in_volume = pd.merge(Ljvf_alpha,volume_mjvf,how="outer",on=["Project ID","J location type","V volume class","F functional class"])

In [ ]:
Ljvf_alpha_in_volume

### Graphs of new relative crashes

In [ ]:
## Graph new relative crashes - model only
(total_crash_change["after_model"] - total_crash_change["before_model"]).plot()

In [ ]:
## Graph new relative crashes - user only
(total_crash_change["after_user"] - total_crash_change["before_user"]).plot()

Yay, these are almost all actually negative now!

In [ ]:
total_crash_change[total_crash_change["after_model"] - total_crash_change["before_model"] > 0]

In [ ]:
## Which ones are positive?
total_crash_change[total_crash_change["after_model"] - total_crash_change["before_model"] > 0].plot(y=["before_model", "after_model"])

In [ ]:
total_crash_change[total_crash_change["after_user"] - total_crash_change["before_user"] > 0].plot(y=["before_user", "after_user"])

Aha, it looks like these projects have a CRF = 1

So crashes are not actually increasing, they are staying the same (and maybe show up as increasing slightly because of rounding?)

In [ ]:
len(total_crash_change[total_crash_change["after_model"] - total_crash_change["before_model"] < 0])/len(total_crash_change)

In [ ]:
len(total_crash_change[total_crash_change["after_model"] - total_crash_change["before_model"] < 0])/len(total_crash_change)

This implies that now the remaining ~40% of results just had a greater increase in volume than decrease in crashes... wait no that can't be right. because this is now scaled by volume. it probably has to do with all the NaN values which I haven't dealt with here yet.

In [ ]:
len(total_crash_change[total_crash_change["after_model"] - total_crash_change["before_model"] < 0]["Project ID"].unique())/len(total_crash_change["Project ID"].unique())

In [ ]:
len(total_crash_change[total_crash_change["after_user"] - total_crash_change["before_user"] < 0]["Project ID"].unique())/len(total_crash_change["Project ID"].unique())

### Graphs of new NC vs old NC results

In [ ]:
## Validate that these are equal?
(existing_crashes["NC_model"]-existing_crashes["NCmojk"]).plot()
## some weird stuff going on here
## Maybe errors with how one or the other was calculated
existing_crashes[abs(existing_crashes["NC_model"]-existing_crashes["NCmojk"]) > 100]
## Look at these projects somehow and try to figure out why they are so off

In [ ]:
(existing_crashes["NC_model"]-existing_crashes["ECmoj with user input"]).plot(figsize = (15,10), ylim=(-25,25))
(existing_crashes["Change_model"]).plot(figsize = (15,10), ylim=(-25,25))
(existing_crashes["Change_user"]).plot(figsize = (15,10), ylim=(-25,25))
## The user-input -> user-input definitely looks the most reasonable - no more than a change in 25 crashes or so

In [ ]:
existing_crashes.plot(y=["ECmoj model","NC_model","ECmoj with user input","NC_user"], figsize = (15,10), ylim=(0,50))

In [ ]:
total_crash_change.plot(y=["Change in crashes","model_crash_change","user_crash_change"],figsize = (15,10))

In [ ]:
## Which projects are now negative? (decrease in crashes)
existing_crashes[existing_crashes["Change_user"] < 0]
## What proportion are now negative? (out of all projects)
len(existing_crashes[existing_crashes["Change_user"] < 0])/len(existing_crashes[existing_crashes["ECmoj with user input"].notna()])

In [ ]:
existing_crashes[existing_crashes["Change_model"] < 0]
len(existing_crashes[(existing_crashes["Change_model"]) < 0])/len(existing_crashes)

In [ ]:
## what proportion was originally negative?
## (just in the original condition where the EC was user input, since that is what this new equation addresses)
existing_crashes[existing_crashes["NCmojk"]-existing_crashes["ECmoj with user input"] < 0]
len(existing_crashes[(existing_crashes["NCmojk"]-existing_crashes["ECmoj with user input"]) < 0])/len(existing_crashes[existing_crashes["ECmoj with user input"].notna()])

How did it get worse somehow??
Maybe this will get better with the relative crash rates, since it probably has to do with the increases in volume (which are now actually being applied to the user based EC, while before they would be applied to the model based EC??)

In [ ]:
## Graph separated by mode and location type - look into this further later, there seems to be big disparities between bicycling roadway and walking roadway etc
existing_crashes.groupby(["J Location","M Mode"])["Change_user"].plot(legend="true",figsize = (15,10))

In [ ]:
existing_crashes[existing_crashes["M Mode"]=="bicycling"].groupby(["J Location"])["Change_user"].plot(legend=True)

In [ ]:
existing_crashes[existing_crashes["M Mode"]=="walking"].groupby(["J Location"])["Change_user"].plot(legend=True)

In [ ]:
existing_crashes.groupby(["J Location","M Mode"])["Change_user"].plot(legend=True)

In [ ]:
existing_crashes[existing_crashes["M Mode"]=="bicycling"].groupby(["J Location"])["Change_model"].plot(legend=True)

In [ ]:
existing_crashes[existing_crashes["M Mode"]=="walking"].groupby(["J Location"])["Change_model"].plot(legend=True)

In [ ]:
existing_crashes.groupby(["J Location","M Mode"])["Change_model"].plot(legend=True)

In [ ]:
existing_crashes["Change_existing"] = existing_crashes["NCmojk"] - existing_crashes["ECmoj with user input"]
existing_crashes[existing_crashes["M Mode"]=="bicycling"].groupby(["J Location"])["Change_existing"].plot(legend=True)

In [ ]:
existing_crashes[existing_crashes["M Mode"]=="walking"].groupby(["J Location"])["Change_existing"].plot(legend=True)

In [ ]:
existing_crashes.groupby(["J Location","M Mode"])["Change_existing"].plot(legend=True)